In [2]:
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn
from numpy.random import uniform
import os
import  numpy as np
import pandas as pd
import math
import random
random.seed(123)


In [3]:
sig_list = [ 
' <Open>', ' <High>', ' <Low>', ' <Close>',
    ' <usc1>',
 ' <usc2>',
 ' <usc3>',
 ' <usc4>',
 ' <usc5>',
 ' <usc6>',
 ' <usc7>',
 ' <usc8>',
 ' <usc9>',
 ' <usc10>',
 ' <usc11>',
 ' <MGapSum 1>',
 ' <MGapSum 2>',
 ' <MGapSum 3>',
 ' <MGapSum 4>',
 ' <MGapSum 5>',
 ' <MGapSum 6>',
 ' <MGapSum 7>',
 ' <MGapSum 8>',
 ' <MGapSum 9>',
 ' <MGapSum 10>',
 ' <MGapSum 11>',
 ' <MGapSum 12>',
 ' <MGapSum 13>',
 ' <MGapSum 14>',
 ' <MGapSum 15>',
 ' <MAvgOfAvg>',
 ' <DblSlwK>',
 ' <DblSlwD>',
 ' <TGapSum 1>',
 ' <TGapSum 2>',
 ' <TGapSum 3>',
 ' <TGapSum 4>',
 ' <TGapSum 5>',
 ' <TGapSum 6>',
 ' <TGapSum 7>',
 ' <TGapSum 8>',
 ' <TGapSum 9>',
 ' <TGapSum 10>',
 ' <TGapSum 11>',
 ' <TGapSum 12>',
 ' <TGapSum 13>',
 ' <TGapSum 14>',
 ' <HA1>',
 ' <HA2>',
 ' <HA3>',
 ' <HA4>',
 ' <HA5>',
 ' <HA6>',
 ' <AvgOfHAs>',
 ' <HA7>',
 ' <HA8>',
 ' <HA9>',
 ' <HA10>',
 ' <HA11>',
 ' <HA12>',
 ' <HA13>',
 ' <HA14>',
 ' <HA15>', ]

In [4]:
'''
specify path to folder
'''
mainDir = 'Hurst/'

In [5]:
def hurst(ts):
    
    lags = range(2, 100)
    tau = [sqrt(std(subtract(ts[lag:], ts[:-lag]))) for lag in lags]

    poly = polyfit(log(lags), log(tau), 1)
    if math.isnan(poly[0]):
        return 0.012 + uniform(0, 0.05)
    return poly[0]*2.0

### try test to ensure code works
gbm = log(cumsum(randn(100000))+1000)
mr = log(randn(100000)+1000)
tr = log(cumsum(randn(100000)+1)+1000)

# Output the Hurst Exponent for each of the above series
print( "Hurst(GBM):   %s" % hurst(gbm) )
print("Hurst(MR):    %s" % hurst(mr) )
print( "Hurst(TR):    %s" % hurst(tr) )



Hurst(GBM):   0.4904455159156204
Hurst(MR):    -2.27511446896674e-06
Hurst(TR):    0.9576881926313228


In [6]:
uniq_tickers = list(set([ m.split(' ')[0] for m in os.listdir(mainDir) ] ))
print(uniq_tickers)

['IGLB', 'NOBL', '$VVIX.X', 'SPY', '.ipynb_checkpoints', '$VIX.X', 'TLT', '@ES']


In [7]:
# ticker_files = {}
# for n in uniq_tickers:
#     ticker_files[n] = sorted( [x for x in os.listdir(mainDir) if n in x ] )

In [8]:
# for k, v in ticker_files.items():
#     print('{0} {1}'.format(k, len(v)))

In [9]:
'''
this code does filtering and transform of data + calculate hurst per file
'''

def filter_volume(df):
    
    if ' <Volume>' in df.columns:
        vol_col = ' <Volume>'
    elif ' <TotalVolume>' in df.columns:
        vol_col = ' <TotalVolume>'
    else:
        raise NameError('Column name does not exist')

    df_clean = df.copy()
    df_clean = df_clean[ df_clean[vol_col] > 0]
    
        
    return df_clean.drop(columns = vol_col)

def filter_un_days(df, filen):
    
    if 'day' in filen.lower():
        
        if '<Date>' in df.columns:
            req_col = '<Date>'
        else:
            raise NameError('Column name does not exist')

        df_clean = df.copy()
        df_clean = df_clean.drop_duplicates(subset=req_col, keep="last")
      
        df_clean = df_clean.drop(columns = ['<Date>'])
     
        return df_clean
    
    else:
        
        #print('here2')
        req_col = '<Date>'
        
        df_clean = df.copy()
        
        df_clean = df_clean.drop_duplicates(subset=req_col, keep="last")
      
        df_clean = df_clean.drop(columns = ['<Date>', ' <Time>'])
       # df_clean.iloc[500 : -500, :]
        return df_clean.iloc[500 : -500, :]
    

def calculate_hurst_per_df(df):
    hurst_dict = {}
    all_columns = []
    for col in sig_list:
        #print(col)
        if col in df.columns:
            
            tau = hurst( np.log( np.array(df[col].values) ))
            hurst_dict[col] = tau
            all_columns.append(col)
#     for k,v in hurst_dict.items():
#         print('{0}  -- {1}'.format(k,v))
    finalDF = pd.DataFrame(list(hurst_dict.values()), index = all_columns)
    
    finalDF['new_col'] = finalDF.index
    finalDF.columns = ['H', 'Feature']
    
    
    return finalDF[['Feature', 'H']]
    

In [10]:
##'main loop'

ticker_files = os.listdir(mainDir)
pathName = '.'
for f in ticker_files:
    print(f)
    curDF = pd.read_csv(mainDir + f, sep = ',')

    modOneDF = filter_volume(curDF)
    modTwoDF = filter_un_days(modOneDF, f)

    finalDF = calculate_hurst_per_df(modTwoDF)
    if not os.path.exists(pathName):
        os.makedirs(pathName)

    finalDF.to_csv(pathName + '/' + f.split('.')[0] + '.csv')
    print(finalDF['H'].dropna().max())
        
       # print('{0} , {1}, {2}, {3}'.format(f , curDF.shape, modOneDF.shape, modTwoDF.shape))

$VIX.X 1 Day106pt.txt


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


LinAlgError: SVD did not converge in Linear Least Squares